In [46]:
import warnings
warnings.filterwarnings("ignore")

In [47]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import preprocessing 
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

In [48]:
df = pd.read_csv("features_30_sec.csv")

In [49]:
df.drop("filename", axis = 1, inplace=True)

In [50]:
df.head()

,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,661794,0.350088,0.088757,0.130228,0.002827,1784.165850,129774.064525,2002.449060,85882.761315,3805.839606,...,52.420910,-1.690215,36.524071,-0.408979,41.597103,-2.303523,55.062923,1.221291,46.936035,blues
1,661794,0.340914,0.094980,0.095948,0.002373,1530.176679,375850.073649,2039.036516,213843.755497,3550.522098,...,55.356403,-0.731125,60.314529,0.295073,48.120598,-0.283518,51.106190,0.531217,45.786282,blues
2,661794,0.363637,0.085275,0.175570,0.002746,1552.811865,156467.643368,1747.702312,76254.192257,3042.260232,...,40.598766,-7.729093,47.639427,-1.816407,52.382141,-3.439720,46.639660,-2.231258,30.573025,blues
3,661794,0.404785,0.093999,0.141093,0.006346,1070.106615,184355.942417,1596.412872,166441.494769,2184.745799,...,44.427753,-3.319597,50.206673,0.636965,37.319130,-0.619121,37.259739,-3.407448,31.949339,blues
4,661794,0.308526,0.087841,0.091529,0.002303,1835.004266,343399.939274,1748.172116,88445.209036,3579.757627,...,86.099236,-5.454034,75.269707,-0.916874,53.613918,-4.404827,62.910812,-11.703234,55.195160,blues


In [51]:
df.shape

(1000, 59)

In [52]:
X = df.drop("label", axis = 1)
y = df["label"]

In [53]:
label_encoder = preprocessing.LabelEncoder() 
y = label_encoder.fit_transform(y) 

In [54]:
len(y)

1000

In [55]:
X_train, y_train, X_test, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Gradient boost

In [56]:
from xgboost import XGBClassifier

In [57]:
model  = XGBClassifier(learning_rate = 0.01)

In [58]:
model.fit(X_train, X_test)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.01, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [64]:
y_pred =  model.predict(y_train)
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
print("Accuracy:", accuracy)
print("Confusion Matrix:\n", conf_matrix)

Accuracy: 0.6533333333333333
Confusion Matrix:
 [[19  0  5  1  2  4  3  0  0  1]
 [ 0 19  0  0  0  0  0  0  1  0]
 [ 5  0 21  1  1  5  0  0  0  4]
 [ 0  0  2 20  7  0  1  1  2  1]
 [ 0  0  0  0 21  0  1  0  2  0]
 [ 2  3  1  1  0 25  0  0  1  0]
 [ 0  0  1  1  2  0 26  0  0  0]
 [ 0  0  0  1  0  0  0 19  3  0]
 [ 1  0  1  1  6  1  0  6 12  1]
 [ 3  0  4  1  1  4  5  1  2 14]]


# Ada boost

In [65]:
from sklearn.ensemble import AdaBoostClassifier

In [66]:
ada_model = AdaBoostClassifier(learning_rate=0.01, random_state=42)

In [67]:
ada_model.fit(X_train, X_test)

AdaBoostClassifier(learning_rate=0.01, random_state=42)

In [68]:
y_pred = ada_model.predict(y_train)
print(accuracy_score(y_test, y_pred))

0.34
